In [1]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Function to calculate distance between two lat-lng points
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    # Distance in kilometers
    distance = R * c
    return distance
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'

In [2]:
df = pd.read_csv('merged_rides__with_elevation.csv')


In [3]:
df['Start_Altitude'] = df[['Start_Altitude', 'Start_Altutude', 'State_Altitude']].bfill(axis=1).iloc[:, 0]
df.drop(columns=['State_Altitude', 'Start_Altutude'], inplace=True)

     

df['End_Altitude'] = df[['End_Altitude', 'end_altitude', 'end_Altitude']].bfill(axis=1).iloc[:, 0]
     

df.drop(columns=['end_Altitude', 'end_altitude'], inplace=True)
     

df['Start_Altitude'].fillna(method='ffill', inplace=True)
df['End_Altitude'].fillna(df['End_Altitude'].mean(), inplace=True)

In [4]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Start_Altitude,End_Altitude,Elevation_Change
0,C2F7DD78E82EC875,electric_bike,1/13/2022 11:59,1/13/2022 12:02,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual,184.1,603.6,419.5
1,A6CF8980A652D272,electric_bike,1/10/2022 8:41,1/10/2022 8:46,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual,184.6,603.6,419.0
2,BD0F91DFF741C66D,classic_bike,1/25/2022 4:53,1/25/2022 4:58,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member,181.7,592.5,410.8
3,CBB80ED419105406,classic_bike,1/4/2022 0:18,1/4/2022 0:33,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual,182.9,598.1,415.2
4,DDC963BFDDA51EEA,classic_bike,1/20/2022 1:31,1/20/2022 1:37,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member,182.5,594.7,412.2


In [ ]:
from sklearn.impute import SimpleImputer
df['Elevation_Change'] = df['End_Altitude'] - df['Start_Altitude']
     

imputer = SimpleImputer(strategy='mean')
df['Elevation_Change'] = imputer.fit_transform(df[['Elevation_Change']])
     

df = df.drop(columns=['Start_Altitude', 'End_Altitude'])
     

df['Distance'] = df.apply(lambda x: haversine_distance(x['start_lat'], x['start_lng'], x['end_lat'], x['end_lng']), axis=1)
     

df = df.drop(columns=['start_lat', 'start_lng', 'end_lat', 'end_lng'])
     

df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
     

df['season'] = df['started_at'].dt.month.map(get_season)
     

def get_day_of_week(timestamp):
    return timestamp.strftime("%A")
     

# Apply the function to each row in the DataFrame
df['day_of_week'] = df['started_at'].apply(get_day_of_week)
     

df['trip_duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
target_variable = 'rideable_type'
y = df[target_variable]
c = ['ride_id', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id',
       'member_casual','TMAX', 'TMIN']
X = df.drop(c, axis=1)
     

label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':  # This identifies categorical columns
        num_unique_values = len(X[column].unique())
        if num_unique_values < 10:  # Limit set for one-hot encoding
            # Apply one-hot encoding to columns with fewer unique values
            dummies = pd.get_dummies(X[column], prefix=column)
            X = pd.concat([X, dummies], axis=1)
        else:
            # Apply label encoding to columns with many unique values to save memory
            label_encoders[column] = LabelEncoder()
            X[column] = label_encoders[column].fit_transform(X[column].astype(str))
        X.drop(column, axis=1, inplace=True)  # Drop original column after encoding
     

Index(['Elevation_Change', 'Distance', 'trip_duration',
       'rideable_type_classic', 'rideable_type_electric', 'season_Fall',
       'season_Spring', 'season_Summer', 'season_Winter', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday'],
      dtype='object')

In [ ]:
y_train

2220921     electric
6189410      classic
587405      electric
10376738     classic
3510934      classic
              ...   
2234489     electric
4304572     electric
10081351    electric
6550634     electric
6423388     electric

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
     

y_train_encoded = (y_train == 'electric').astype(int)  # Encode 'electric' as 1, 'classic' as 0
y_test_encoded = (y_test == 'electric').astype(int)
     

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
     

model = LinearRegression()

     

X_train.columns

Index(['Elevation_Change', 'Distance', 'trip_duration', 'season_Fall',
       'season_Spring', 'season_Summer', 'season_Winter', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday'],
      dtype='object')

In [ ]:
X_train.drop(columns=['ended_at'], inplace=True)
     

print(X_train.dtypes)
print(y_train_encoded)
     

Elevation_Change         float64
Distance                 float64
trip_duration            float64
season_Fall                 bool
season_Spring               bool
season_Summer               bool
season_Winter               bool
day_of_week_Friday          bool
day_of_week_Monday          bool
day_of_week_Saturday        bool
day_of_week_Sunday          bool
day_of_week_Thursday        bool
day_of_week_Tuesday         bool
day_of_week_Wednesday       bool
dtype: object
2220921     1
6189410     0
587405      1
10376738    0
3510934     0
           ..
2234489     1
4304572     1
10081351    1
6550634     1
6423388     1
Name: rideable_type, Length: 7971288, dtype: int64

LogisticRegression

In [ ]:
X_important = X[['Distance', 'Elevation_Change', 'trip_duration']]

y = df[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X_important, y, test_size=0.3, random_state=42)

In [ ]:
y_pred = model.predict(X_test)
y_pred_encoded = (y_pred > 0.5).astype(int)

mse = mean_squared_error(y_test_encoded, y_pred)
r2 = r2_score(y_test_encoded, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

coefficients = pd.DataFrame(model.coef_.T, X_train.columns, columns=['Coefficient'])
print(coefficients)

Mean Squared Error: 0.4424835646628323
R² Score: -0.7710250423778622
                       Coefficient
Elevation_Change         -0.000671
Distance                  0.000095
trip_duration            -0.014597
season_Fall               0.083438
season_Spring             0.069393
season_Summer             0.001189
season_Winter             0.043723
day_of_week_Friday        0.094066
day_of_week_Monday       -0.004058
day_of_week_Saturday     -0.002810
day_of_week_Sunday       -0.015210
day_of_week_Thursday      0.065395
day_of_week_Tuesday       0.009808
day_of_week_Wednesday     0.050551

In [ ]:
recall = recall_score(y_test_encoded, y_pred_encoded)  # pos_label defaults to 1 which is suitable here
f1 = f1_score(y_test_encoded, y_pred_encoded)
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# For a comprehensive report including precision and support per class
report = classification_report(y_test_encoded, y_pred_encoded)
print(report)
     

Recall: 0.7923400122820297
F1-Score: 0.6472793947448984
              precision    recall  f1-score   support

           0       0.59      0.31      0.41   1665742
           1       0.55      0.79      0.65   1750525

    accuracy                           0.56   3416267
   macro avg       0.57      0.55      0.53   3416267
weighted avg       0.57      0.56      0.53   3416267